# NBA Game Prediction

This project is developed for predicting upcoming NBA games outcomes (W/L) using Machine Learning algorithms. For data gathering nba_api was used.

**Features used in the ML algorithms**

> Current season wins - how many games have been won this season before the observable game

> Current season losses - how many games have been lost this season before the observable game

> Previous season wins - sum of previous season wins

> Previous season losses - sum of previous season losses

> Season before last wins - sum of season before last wins

> Season before last losses - sum of season before last losses

> Home or away (1 - home, 0 - away) - if the observable game was played at home or away

**Mounting to Drive**

In [1]:
from google.colab import drive

# Mounting to GD
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/NBA prediction/')

Mounted at /content/drive


**Imports**

In [2]:
!pip install nba_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 4.4 MB/s eta 0:00:00


In [3]:
# Pandas
import pandas as pd

# NBA api
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import cumestatsteamgames
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.library.parameters import Season
from nba_api.stats.endpoints import winprobabilitypbp

# Datetime
from datetime import date
from datetime import timedelta

# Time
import time

# sklearn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics  import f1_score,accuracy_score

# **Project Ideas**

**Data from somewhere the mid 80s is not usable (missing values)**

**See: [link](https://docs.google.com/spreadsheets/d/1J5Fk9F7-X7iiZWb-p9J5eHV_80dn7uD627ZNCfajTyY/edit#gid=724387784)**

**Previous Years Conference Ranking?? (Can be found under nba_api.teamyearbyyearstat)**

**Home or away team can be found under nba_api.winprobabilitypbp (need to clarify available years)**

**Features to pick**

wins this season (home and away) 	 | losses this season (home and away)  |

last 2 year wins (home and away)     | last 2 year losses (home and away)  |

previous year wins (home and away)   | previous year losses (home and away)|

how many mvp players (home and away) | ranking this season (home and away) |

home or away game (home and away) |  

In [4]:
# Google Sheet keys and names where the data is held

gsheetkeys = ["1lv2-3WhrFHvNL1X5S7b-ipqARNKaaOkT0-DJPSwscs8", "1b7lS4X07tYRY_KCXZS7i6JQX7hY8vUnLHUZ1JRmAeLo",
              "1xPsisR8NMg8coNMf0_3VipPAWKAdW89phLL7S5dKFqs", "1JOADxIyhpNDdO3gmY7aNReXUgqNZRa_ZBY_GR-oT-K4", "1j70bhwPsO-44BVlXuMWMwgvYH8p3iYsJbUw4VELrkgI",
              "1mtAjjWnObMejnF4o26ATQwH0vwG58clnk8HBZyFbACQ",
              "1yAO9YJ2o5hUVfuqjSKgycREhyVq-3lU-Eu2dZftQUtk",
              "1wN2cWd9pW0oHVX156CtMVES47fYhlsL-QvUplL5l91c", "10rcJFymCTL2ngfzSpS79dcpzvOe5oocZjC-78e9Wl74", "1p0AnDfGH_UWQzdzuVGvpegoa9qq6y1cZnkwXMGwOfUM",
              "1DnScQ8Vm8-kTQAlcRUagqG90yEGMqf5__gp9PmO-IEE", "1Elz4-Zy98L_6fINcqpWS5s6EW3s9oTs5FJmDKo5ihDA", "1hFRQTFdxpJK1LwYGEFJ4LOgAWIf0igKM2iymJxGlGrg",
              "1_dGCmA-zGJ7MhDrQNlV_IoTMDL8w_qTgttyUpBseeRc", "1-nAx4To5U4_9kK9m76xGm1NZC7fvzi42LV8LUrQDRg0", "1EVLOnUqjov0J8KAjIHhi18mTvE95HGaurZ45qa8izEk",
              "1DHKZV_rVgrQSjoJXs8YQC0Ku5ECjoWXOEhPaQaf6wCE", "1S5gKj8fXmtBaoGTlEaWBb0z3Zvvk0WKhyBmlffNSbu8", "1t8ooshG7ixX-FXR8RRY-kOdCYXvNhUQHZv3iip7ldfw",
              "1frY1JfGAcrzD_UiO1TIY65yTg_dVXTI4sEvWARnTta0", "1bKF8JivyCT46wzgCiy3wFqgJ0kud26n3VMeROHudm8E", "1HwfT9T4vs8ddn4XjCGONX8uKsUxgeiGoMFsXG8ShBFE",
              "1VClP32CCKtrKFEV56Y9YzcC7ApbyKTQ555-k8dC-j1c", "1qdPrE92kWcFTGPrFi-XLOR2hBa1_pZrpsIMQVcLeZYA", "1efI02TVgiKmNiIW6soOMf7H51XQ3_ycH5Mj3ExYm6Xk",
              "1gojRc7Igdpw6M3TVesN4is-PB7924Cv_J_bWt6MNEb0", "1q9ve_zN2j5XEzMkAahEp2N5y6eqZFt4wIXasxIVlX7A", "1RfbIcqSXG1Eh4vfRwnr1kg_rhtYofciYTHa1r5G46WM",
              "15hhZmljZDq1QYNshoX5_auqg7tiP9VrPfeNXYOK_sBg", "1LuORJk4XPerSDPMlmaWwiU_9ZUmvj3QktMEVUqKQ4YI", "1PHXdA7QcQPqgu9bEOoarNPbvMEOWhGPnlQXZtTM6eqc",
              "1rJ46fT86RJYrSK5weR0qnQjqL9MLPKvy7dUaukPuqqA", "1KXYUr3mnuWIGa66LHjkrvDYbqSwx2_r9mWf2Lpa6IB8", "1BmSDiRfm9QQqDfd33FUUlixHSrxjzzlEjhGv-5V1iuo"]
sheet_names = ["2023_nba_games_testing", "2022_nba_games_testing", "2021_nba_games_testing", "2020_nba_games_testing", "2019_nba_games_testing",
               "2018_nba_games_testing",
               "2017_nba_games_testing",
               "2016_nba_games_testing", "2015_nba_games_testing", "2014_nba_games_testing", "2013_nba_games_testing", "2012_nba_games_testing",
               "2011_nba_games_testing", "2010_nba_games_testing", "2009_nba_games_testing", "2008_nba_games_testing", "2007_nba_games_testing",
               "2006_nba_games_testing", "2005_nba_games_testing", "2004_nba_games_testing", "2003_nba_games_testing", "2002_nba_games_testing",
               "2001_nba_games_testing", "2000_nba_games_testing", "1999_nba_games_testing", "1998_nba_games_testing", "1997_nba_games_testing",
               "1996_nba_games_testing", "1995_nba_games_testing", "1994_nba_games_testing", "1993_nba_games_testing", "1992_nba_games_testing",
               "1991_nba_games_testing", "1990_nba_games_testing"]

In [6]:
# Reading the data from Google Sheets

data = pd.DataFrame()

for i in range(len(sheet_names)):
    gsheetkey = gsheetkeys[i]
    sheet_name = sheet_names[i]
    url = f'https://docs.google.com/spreadsheet/ccc?key={gsheetkey}&output=xlsx'
    try:
        sheet_data = pd.read_excel(url, sheet_name=sheet_name)
    except RuntimeError:
        continue
    data = pd.concat([data, sheet_data])

In [7]:
data.drop(["Unnamed: 0"], axis=1, inplace=True)

In [12]:
# Needed for picking the same amount
# of positive and negative cases

data["game_outcome"].value_counts()

1    6178
0    5892
Name: game_outcome, dtype: int64

In [10]:
# Creating train and test sets

negative=data[data['game_outcome']==0]
#positive=data[data['game_outcome']==1]
#negative=data[data['game_outcome']==0].sample(n=positive.shape[0])
positive=data[data['game_outcome']==1].sample(n=negative.shape[0])
final=pd.concat([positive,negative])
final = final.sample(frac=1)

y = final["game_outcome"]
final.drop(["game_outcome"], axis=1, inplace=True)
X = final

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

In [11]:
final.head()

,current_season_wins,current_season_losses,previous_season_wins,previous_season_losses,season_before_last_wins,season_before_last_losses,home_or_away
132,8,4,61,21,42,40,1
181,45,34,35,55,30,57,1
78,0,1,30,52,36,46,0
163,13,25,19,63,15,67,1
270,18,37,29,53,25,57,1


# **AI models**

**Neural Network**

In [13]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(32, 32, 32, 32, 32, 32, 32, 32), activation='relu', solver='adam', max_iter=15000)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

In [14]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

[[3163 1217]
 [1162 3296]]
              precision    recall  f1-score   support

           0       0.73      0.72      0.73      4380
           1       0.73      0.74      0.73      4458

    accuracy                           0.73      8838
   macro avg       0.73      0.73      0.73      8838
weighted avg       0.73      0.73      0.73      8838



In [ ]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[826 670]
 [586 864]]
              precision    recall  f1-score   support

           0       0.58      0.55      0.57      1496
           1       0.56      0.60      0.58      1450

    accuracy                           0.57      2946
   macro avg       0.57      0.57      0.57      2946
weighted avg       0.57      0.57      0.57      2946



**XGBoost Classifier**

In [15]:
from xgboost.sklearn import XGBClassifier

XGB = XGBClassifier()
XGB.fit(X_train,y_train)

pred=XGB.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.5902919212491514
0.5942857142857143


**LGBM Classifier**

In [16]:
from lightgbm import LGBMClassifier

LGBM = LGBMClassifier()
LGBM.fit(X_train,y_train)

pred=LGBM.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

[LightGBM] [Info] Number of positive: 4458, number of negative: 4380
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000567 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 8838, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504413 -> initscore=0.017652
[LightGBM] [Info] Start training from score 0.017652
0.59979633401222
0.6073926073926075


**Gradient Boosting Classifier**

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier()
GB.fit(X_train,y_train)

pred=GB.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.6130346232179226
0.6143437077131259


**Random Forest**

In [18]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()
RF.fit(X_train,y_train)

pred=RF.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.5855397148676171
0.5811320754716982


**Decision Tree**

In [19]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(X_train,y_train)

pred=DT.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.5492192803801765
0.5482993197278911


**KNN**

In [20]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier()
KNN.fit(X_train,y_train)

pred=KNN.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.548540393754243
0.5445205479452054


**Stochastic Gradient Descent Classifier**

In [21]:
from sklearn.linear_model import SGDClassifier

SGDC = SGDClassifier()
SGDC.fit(X_train,y_train)

pred=SGDC.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.5071283095723014
0.65738555922605


**Naive Bayes**

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

GNB = GaussianNB()
MNB = MultinomialNB()

GNB.fit(X_train,y_train)
MNB.fit(X_train,y_train)


pred_GNB=GNB.predict(X_test)
pred_MNB=MNB.predict(X_test)

print(accuracy_score(y_test,pred_GNB))
print(f1_score(y_test,pred_GNB))
print(accuracy_score(y_test,pred_MNB))
print(f1_score(y_test,pred_MNB))

0.5987780040733197
0.604946524064171
0.5916496945010183
0.5977933801404214


**Support Vector Machine**

In [23]:
from sklearn.svm import SVC

SVM = SVC()
SVM.fit(X_train,y_train)

pred=SVM.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.5964019008825526
0.6077202243483999


**Logistic Regression**

In [24]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train,y_train)

pred=LR.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.6137135098438561
0.6199064796259186


**DecisionTreeClassifier**

In [25]:
DT = DecisionTreeClassifier()
DT.fit(X_train,y_train)

pred=DT.predict(X_test)

print(accuracy_score(y_test,pred))
print(f1_score(y_test,pred))

0.5522742701968771
0.5456424388563555


# **Data Reading**

In [26]:
# Creating a dictionary of team abbreviations
# and team id's

nba_teams = teams.get_teams()
teams_abbreviations = {}

for team in nba_teams:
    teams_abbreviations[team.get("abbreviation")] = str(team.get("id"))

print(teams_abbreviations)

{'ATL': '1610612737', 'BOS': '1610612738', 'CLE': '1610612739', 'NOP': '1610612740', 'CHI': '1610612741', 'DAL': '1610612742', 'DEN': '1610612743', 'GSW': '1610612744', 'HOU': '1610612745', 'LAC': '1610612746', 'LAL': '1610612747', 'MIA': '1610612748', 'MIL': '1610612749', 'MIN': '1610612750', 'BKN': '1610612751', 'NYK': '1610612752', 'ORL': '1610612753', 'IND': '1610612754', 'PHI': '1610612755', 'PHX': '1610612756', 'POR': '1610612757', 'SAC': '1610612758', 'SAS': '1610612759', 'OKC': '1610612760', 'TOR': '1610612761', 'UTA': '1610612762', 'MEM': '1610612763', 'WAS': '1610612764', 'DET': '1610612765', 'CHA': '1610612766'}


In [27]:
# Finds the given team abbreviation team id
#
# Return: team_id

def FindTeamId(team_abbreviation):
    return teams_abbreviations.get(team_abbreviation)

In [28]:
# Finds matches of a given team
#
# Return: matches (Pandas df)

def FindTeamMatches(team_id):
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_type_nullable=SeasonType.regular)
    games = gamefinder.get_data_frames()[0]
    return games

In [30]:
# Finds the wins and losses for a team in the given season
#
# Return: wins, losses

def SeasonWinsAndLosses(team_abbreviation, season):
    wins = 0
    losses = 0

    # Finds all of the matches of a given team
    team_matches = FindTeamMatches(FindTeamId(team_abbreviation))
    team_matches = team_matches[team_matches["SEASON_ID"].str[-4:] == season]

    for index, row in team_matches.iterrows():
        if (row["WL"] == "W"):
            wins += 1
        else:
            losses += 1

    return wins, losses

# Finds the wins and losses for a team in the given season
# until the given date (for searching ongoing season games)
#
# Return: wins, losses

def SeasonWinsAndLossesWithDate(team_abbreviation, season, until_date):
    wins = 0
    losses = 0

    # Finds all of the matches of a given team on the given season which are before until_date
    team_matches = FindTeamMatches(FindTeamId(team_abbreviation))
    team_matches = team_matches[team_matches["SEASON_ID"].str[-4:] == season]
    team_matches['GAME_DATE'] = pd.to_datetime(team_matches['GAME_DATE'], format='%Y-%m-%d')
    team_matches = team_matches.loc[team_matches['GAME_DATE'] < until_date]

    for index, row in team_matches.iterrows():
        if (row["WL"] == "W"):
            wins += 1
        else:
            losses += 1


    return wins, losses

In [31]:
# Finds the nickname of a team with the given team_id
#
# Return: Team nickname

def FindTeamNickname(team_id):
    all_teams = teams.get_teams()
    found = [team for team in all_teams if str(team["id"]) == team_id][0]
    team_nickname = found["nickname"]

    return team_nickname

In [32]:
# Finds if the given team was on home court or away at the given game
#
# Return: 1 - home court, 0 - away court

def FindHomeOrAway(team_abbreviation, game_id):
    home_or_away = 0

    game = winprobabilitypbp.WinProbabilityPBP(game_id=game_id, run_type=winprobabilitypbp.RunType.default)
    game_data = game.game_info.data.get("data")[0]

    home_team_id = str(game_data[2])
    team_id = teams_abbreviations.get(team_abbreviation)

    if (home_team_id == team_id):
        home_or_away = 1

    return home_or_away

In [33]:
# Finds the first game between given dates
# also finds if it was a home game or an away game
#
# Return (outcome): 1 - game was a win, 0 - game was a loss
# Return (home_or_away): 1 - home game, 0 - away game

def FindGameOutcome(team_abbreviation, season, start_date, end_date):
    outcome = 0

    team_matches = FindTeamMatches(FindTeamId(team_abbreviation))
    team_matches = team_matches[team_matches["SEASON_ID"].str[-4:] == season]
    team_matches['GAME_DATE'] = pd.to_datetime(team_matches['GAME_DATE'], format='%Y-%m-%d')
    team_matches = team_matches.loc[(team_matches['GAME_DATE'] >= start_date) &
                                    (team_matches['GAME_DATE'] <= end_date)]

    team_matches = team_matches.sort_values(by='GAME_DATE')

    if (len(team_matches) == 0):
        return None, None

    first_match = team_matches.iloc[0]
    game_id = str(first_match["GAME_ID"])

    if (first_match["WL"] == "W"):
        outcome = 1

    home_or_away = FindHomeOrAway(team_abbreviation, game_id)

    return outcome, home_or_away


In [34]:
# To be implemented

#def FindNextGameOutcome(team_abbreviation, team_matches, season)
#game_outcome, home_or_away = FindGameOutcome(abbreviation, str(season), until_date, AddXDaysToTheDate(until_date, addition))

In [ ]:
# Takes a string date and adds days to it
#
# Return: string date with the added days

def AddXDaysToTheDate(given_date, addition):
    date_obj = date.fromisoformat(given_date)
    time_delta = timedelta(days=addition)
    date_obj = date_obj + time_delta
    changed_date = date.isoformat(date_obj)
    return changed_date

In [ ]:
# This is code for collecting data by teams.
# This data is to be used in the future for the Machine Learning algorithms.
# With this approach more data can be collected, because
# all of the games found for every team is gathered.

end_season = 2022

# 2000 - 2022 interpreted teams = ["ATL", "BOS", "CLE"]
# 2010 - 2022 interpreted teams = ["DAL", "CHI", "NOP"]

for key in teams_abbreviations.keys():
    if (key == "ATL" or key == "BOS" or key == "CLE" or key == "DAL" or key == "CHI" or key == "NOP"):
        continue
    final_df = pd.DataFrame()
    season = 2010
    while season <= end_season:
        print(key, season)
        try:
            season_before_last_wins, season_before_last_losses = SeasonWinsAndLosses(key, str(season - 2))
            if (season_before_last_wins == 0 or season_before_last_losses == 0):
                season += 1
                time.sleep(1)
                continue
            time.sleep(1)

            previous_season_wins, previous_season_losses = SeasonWinsAndLosses(key, str(season - 1))
            if (previous_season_wins == 0 or previous_season_losses == 0):
                season += 1
                time.sleep(1)
                continue
            time.sleep(1)

            current_season_wins = []
            current_season_losses = []
            game_outcomes = []
            home_or_aways = []

            team_matches = FindTeamMatches(teams_abbreviations.get(key))
            time.sleep(1)
            team_matches = team_matches[team_matches["SEASON_ID"].str[-4:] == str(season)]
            team_matches = team_matches.sort_values(by='GAME_DATE')
            i = 0

            for index, row in team_matches.iterrows():
                game_outcome = 0
                game_id = str(row["GAME_ID"])
                home_or_away = FindHomeOrAway(key, game_id)
                time.sleep(0.6)

                home_or_aways.append(home_or_away)

                if (row["WL"] == "W"):
                    game_outcome = 1

                if (i == len(team_matches) - 1):
                    game_outcomes.append(game_outcome)
                    break

                if (i == 0):
                    current_season_wins.append(0)
                    current_season_losses.append(0)
                    if (game_outcome == 0):
                        current_season_losses.append(current_season_losses[len(current_season_losses) - 1] + 1)
                        current_season_wins.append(current_season_wins[len(current_season_wins) - 1])
                    else:
                        current_season_losses.append(current_season_losses[len(current_season_losses) - 1])
                        current_season_wins.append(current_season_wins[len(current_season_wins) - 1] + 1)
                else:
                    if (game_outcome == 0):
                        current_season_losses.append(current_season_losses[len(current_season_losses) - 1] + 1)
                        current_season_wins.append(current_season_wins[len(current_season_wins) - 1])
                    else:
                        current_season_losses.append(current_season_losses[len(current_season_losses) - 1])
                        current_season_wins.append(current_season_wins[len(current_season_wins) - 1] + 1)

                i += 1
                game_outcomes.append(game_outcome)

        except TimeoutError:
            time.sleep(5)
            print("TIMEOUTERROR")
            continue

        season += 1

        df = pd.DataFrame()
        df["current_season_wins"] = current_season_wins
        df["current_season_losses"] = current_season_losses
        df["previous_season_wins"] = previous_season_wins
        df["previous_season_losses"] = previous_season_losses
        df["season_before_last_wins"] = season_before_last_wins
        df["season_before_last_losses"] = season_before_last_losses
        df["game_outcome"] = game_outcomes
        df["home_or_away"] = home_or_aways

        out_file = f"{key}_nba_games_data.csv"
        df.to_csv(out_file, sep="\t", encoding="utf-8", mode="a", header=False)
    break

In [ ]:
# This is code for collecting data by seasons
# Currently this data is used on the Machine Learning algorithms

# Season goes roughly from October to April

# Covered seasons: 2002-2022, 1992-2002, 1982-1992

season = 2019
end_season = 2023

addition = 12

out_file = "{0}_nba_games_testing.csv".format(season)

all_teams = teams.get_teams()
all_teams_abbreviations = [team["abbreviation"] for team in all_teams]

i = 0

while season <= end_season:
    final_df = pd.DataFrame()
    #until_date = str(season) + "-11-01"
    final_date = date.fromisoformat(str(season + 1) + "-04-01")
    print(season)

    for abbreviation in all_teams_abbreviations:
        until_date = str(season) + "-11-01"
        #if (i == 1):
        #    break
        try:
            season_before_last_wins, season_before_last_losses = SeasonWinsAndLosses(abbreviation, str(season - 2))
            if (season_before_last_wins == 0 or season_before_last_losses == 0):
                continue
            time.sleep(0.6)

            previous_season_wins, previous_season_losses = SeasonWinsAndLosses(abbreviation, str(season - 1))
            if (previous_season_wins == 0 or previous_season_losses == 0):
                continue
            time.sleep(0.6)

            current_season_wins = []
            current_season_losses = []
            game_outcomes = []
            home_or_aways = []

            while (date.fromisoformat(until_date) < final_date):
                #time.sleep(0.6)
                try:
                    game_outcome, home_or_away = FindGameOutcome(abbreviation, str(season), until_date, AddXDaysToTheDate(until_date, addition))
                except JSONDecodeError:
                    until_date = AddXDaysToTheDate(until_date, addition)
                    continue

                if (game_outcome == None):
                    until_date = AddXDaysToTheDate(until_date, addition)
                    continue

                current_season_win, current_season_loss = SeasonWinsAndLossesWithDate(abbreviation, str(season), until_date)
                time.sleep(0.6)
                #home_or_away = FindHomeOrAway(str(FindTeamId(abbreviation)), FindGameOutcome(abbreviation, str(season), until_date, AddXDaysToTheDate(until_date, addition)), season)
                #print(home_or_away)

                current_season_wins.append(current_season_win)
                current_season_losses.append(current_season_loss)
                game_outcomes.append(game_outcome)
                home_or_aways.append(home_or_away)

                until_date = AddXDaysToTheDate(until_date, addition)
        except TimeoutError:
            time.sleep(15)
            continue

        df = pd.DataFrame()
        df["current_season_wins"] = current_season_wins
        df["current_season_losses"] = current_season_losses
        df["previous_season_wins"] = previous_season_wins
        df["previous_season_losses"] = previous_season_losses
        df["season_before_last_wins"] = season_before_last_wins
        df["season_before_last_losses"] = season_before_last_losses
        df["game_outcome"] = game_outcomes
        df["home_or_away"] = home_or_aways

        #final_df = final_df.append(df)
        final_df = pd.concat([final_df, df], axis=0)
        #i += 1

    season += 1
    final_df.to_csv(out_file, sep="\t", encoding='utf-8')
    out_file = "{0}_nba_games_testing.csv".format(season)